# PDIoT Group A1 model

### Imports

In [1]:
import os
import numpy as np

# ml
import tensorflow as tf

# data
from ml.config import cfg
from ml.dataset import odgt2data
from ml.models import ModelBuilder, OptimizerBuilder, LRScheduleBuilder

# logging
from tqdm import tqdm

print("All packages imported!")

2023-11-09 15:13:39.771146: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


All packages imported!


### Config

In [2]:
config_fp = '/home/jonathan/git/pdiot-ml/cfg/log-motion.yaml'
cfg.merge_from_file(config_fp)
cfg.DATASET.path = '/home/jonathan/git/pdiot-ml/data/sets/pdiot-data/'
cfg.TRAIN.path = '/home/jonathan/git/pdiot-ml/ckpt/log-motion/'    

### Data

In [3]:
train_odgt_fp = os.path.join(cfg.DATASET.path, cfg.DATASET.LIST.train)
val_odgt_fp = os.path.join(cfg.DATASET.path, cfg.DATASET.LIST.val)

train_X, train_y = odgt2data(train_odgt_fp, cfg.MODEL.INPUT.window_size, cfg.TRAIN.DATA.overlap_size)
val_X, val_y = odgt2data(val_odgt_fp, cfg.MODEL.INPUT.window_size, cfg.TRAIN.DATA.overlap_size)

### Model

In [4]:
model = ModelBuilder.build_classifier(cfg.MODEL, '', num_classes=2)
optimizer = OptimizerBuilder.build_optimizer(cfg.TRAIN.OPTIM)
loss = tf.keras.losses.SparseCategoricalCrossentropy()

lr_scheduler = LRScheduleBuilder.build_scheduler(cfg.TRAIN.LR)
lr_callback = tf.keras.callbacks.LearningRateScheduler(lr_scheduler)

early_stop_callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=cfg.TRAIN.LEN.early_stop)

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=os.path.join(cfg.TRAIN.path, 'weights.hdf5'),
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])
history = model.fit(train_X, train_y, validation_data=(val_X, val_y), 
                    epochs=cfg.TRAIN.LEN.num_epoch, 
                    batch_size=cfg.TRAIN.DATA.batch_size, 
                    initial_epoch=cfg.TRAIN.LEN.start_epoch,
                    callbacks=[lr_callback, 
                               early_stop_callback,
                               checkpoint_callback])

Epoch 1/100


2023-11-09 15:13:45.134835: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-09 15:13:45.137015: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


89/89 [==============================] - 1s 10ms/step - loss: 1.6123 - accuracy: 0.6117 - val_loss: 1.0104 - val_accuracy: 0.7483 - lr: 0.0010
Epoch 2/100
89/89 [==============================] - 1s 8ms/step - loss: 1.1127 - accuracy: 0.7498 - val_loss: 1.1809 - val_accuracy: 0.6755 - lr: 0.0010
Epoch 3/100
89/89 [==============================] - 0s 4ms/step - loss: 0.8712 - accuracy: 0.8097 - val_loss: 1.0180 - val_accuracy: 0.8079 - lr: 0.0010
Epoch 4/100
89/89 [==============================] - 1s 7ms/step - loss: 1.0133 - accuracy: 0.8159 - val_loss: 0.8684 - val_accuracy: 0.8013 - lr: 0.0010
Epoch 5/100
89/89 [==============================] - 0s 5ms/step - loss: 0.8325 - accuracy: 0.8447 - val_loss: 1.5945 - val_accuracy: 0.7351 - lr: 0.0010
Epoch 6/100
89/89 [==============================] - 0s 5ms/step - loss: 1.0610 - accuracy: 0.8186 - val_loss: 1.0102 - val_accuracy: 0.8477 - lr: 0.0010
Epoch 7/100
89/89 [==============================] - 1s 7ms/step - loss: 0.8885 - accur

In [6]:
model.load_weights(os.path.join(cfg.TRAIN.path, 'weights.hdf5'))

In [5]:
history.model